https://www.kaggle.com/gogo827jz/inference-demo-notebook

# Code Section

# Sandbox Section

In [31]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/optiver-realized-volatility-prediction/sample_submission.csv
/kaggle/input/optiver-realized-volatility-prediction/train.csv
/kaggle/input/optiver-realized-volatility-prediction/test.csv
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=97/888f813404d8417ca8d6b8aebd5f2951.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=43/bb0efa57f511470e817880842e3e2afa.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=21/1d8dc18ebfee47ffbb54b04e6afc0634.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=72/60f62a03d8854605901dda072c84db39.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=4/761268d671f9429abb29d9d2895e9bd2.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=112/cd283097a5b54293ba400a19e811a7f9.parquet
/kaggle/input/optiver-realized-volatility-pr

In [32]:
import pandas as pd
pd.set_option('display.width', 1000)
df = pd.read_csv('../input//optiver-realized-volatility-prediction/sample_submission.csv')
df.head()

,row_id,target
0,0-4,0.003048
1,0-32,0.003048
2,0-34,0.003048


In [33]:
df = pd.read_csv('../input//optiver-realized-volatility-prediction/train.csv')
df.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [34]:
df = pd.read_csv('../input//optiver-realized-volatility-prediction/test.csv')
df.head()

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [35]:
import glob
list_book_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')
print(list_book_test)

['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0']


In [36]:
pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0/7832c05caae3489cbcbbb9b02cf61711.parquet', engine='pyarrow')

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15


In [37]:
list_trade_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/*')
print(list_trade_test)

['/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0']


In [38]:
pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0/31c83a67d81349208e7d5eace9dbbac8.parquet', engine='pyarrow')

,time_id,seconds_in_bucket,price,size,order_count
0,4,7,1.000344,1,1
1,4,24,1.000049,100,7
2,4,27,1.000059,100,3


# <font size = 4 color = 'yellow'> Reading the book_test file. PS : book_test is provided for stock_id = 0 only.

In [39]:
list_order_book_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')
print(f"list_order_book_file_test \t\t: {list_order_book_file_test}, \ntype(list_order_book_file_test) \t: {type(list_order_book_file_test)}")

list_order_book_file_test = sorted(list_order_book_file_test, key = lambda x: x.split('=')[1])
print(f"list_order_book_file_test \t\t:Sorted {list_order_book_file_test}, \ntype(list_order_book_file_test) \t: {type(list_order_book_file_test)}")

list_order_book_file_test 		: ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0'], 
type(list_order_book_file_test) 	: <class 'list'>
list_order_book_file_test 		:Sorted ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0'], 
type(list_order_book_file_test) 	: <class 'list'>


In [40]:
func_print = True

In [41]:
from tqdm.auto import tqdm
import numpy as np
np.set_printoptions(linewidth=100000)


def log_return(list_stock_prices):
    if func_print : print(f'\n\nInside function log_return \nParameters : list_stock_prices =\n{list_stock_prices} \n')
    
    ret_value = np.log(list_stock_prices).diff()
    if func_print : print(f'Returning from log_return. ret_value = np.log(list_stock_prices).diff() : \n{ret_value}')
    
    return np.log(list_stock_prices).diff()

def realized_volatility(series_log_return):
    if func_print : print(f'\n\nInside function realized_volatility \nParameters : series_log_return =\n {series_log_return} \n')
    ret_value = np.sqrt(np.sum(series_log_return ** 2))
    if func_print : print(f'Returning from function realized_volatility \nret_value = np.sqrt(np.sum(series_log_return ** 2)) : \nret_value = {ret_value} \n\n')
    return np.sqrt(np.sum(series_log_return ** 2))

def calculate_wap(df):
    if func_print : print(f'\n\nInside function calculate_wap \nParameters : df =\n {df} \n')
    a1 = df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']
    a2 = df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']
    b = df['bid_size1'] + df['ask_size1'] + df['bid_size2']+ df['ask_size2']
    if func_print : print(f'Return from function calculate_wap \n\n')
    return (a1 + a2)/ b

def realized_volatility_per_time_id(file_path, prediction_column_name):
    if func_print : print(f'\n\nInside function realized_volatility_per_time_id \nParameters : file_path = {file_path}, prediction_column_name: {prediction_column_name} \n')
    
    df_book_data = pd.read_parquet(file_path)
    if func_print : print(f'df_book_data before calculate_wap \n: {df_book_data} \n')
    
    df_book_data['wap'] = calculate_wap(df_book_data)
    if func_print : print(f'df_book_data after calculate_wap \n: {df_book_data} \n')
    
    df_book_data_grouped = df_book_data.groupby(['time_id'])['wap']
    
    if func_print : print('\nPrinting df_book_data.groupby([''time_id''])[''wap''] in a loop')
    for key, item in df_book_data_grouped:
        if func_print : print(f'key : {key}')
        if func_print : print(f'type(item) : {type(item)}')
        if func_print : print(f'item : {item} \n')
        #print(df_book_data_grouped.get_group(key))
    
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    if func_print : print(f'Back to function realized_volatility_per_time_id')
    if func_print : print(df_book_data)
    
    
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    if func_print : print('\ndf_book_data after removing null log_return rows')
    if func_print : print(df_book_data)
    
    if func_print : print('\nPrinting df_book_data.groupby([''time_id''])[''log_return''] in a loop')
    for key, item in df_book_data.groupby(['time_id'])['log_return']:
        if func_print : print(f'key : {key}')
        if func_print : print(f'type(item) : {type(item)}')
        if func_print : print(f'item : {item} \n')
    
    
    
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    if func_print : print(f'Back to function realized_volatility_per_time_id')
    if func_print : print(f'df_realized_vol_per_stock :\n{df_realized_vol_per_stock}')
    
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    if func_print : print(f'renamed column log_return. \ndf_realized_vol_per_stock :\n{df_realized_vol_per_stock} \n')
    
    stock_id = file_path.split('=')[1]
    if func_print : print(f'Evaluating stock_id = file_path.split(\'=\')[1]. \nfile_path : {file_path}\nstock_id : {stock_id} \n')
    
    if func_print : print(f'df_realized_vol_per_stock :\n{df_realized_vol_per_stock} \n')
    if func_print : print('Evaluating df_realized_vol_per_stock[\'row_id\'] = df_realized_vol_per_stock[\'time_id\'].apply(lambda x:f\'{stock_id}-{x}\')')
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    if func_print : print(f'df_realized_vol_per_stock :\n{df_realized_vol_per_stock}')
    
    ret_value = df_realized_vol_per_stock[['row_id', prediction_column_name]]
    if func_print : print('Returning from function realized_volatility_per_time_id.')
    if func_print : print('Return Expr : ret_value = df_realized_vol_per_stock[[\'row_id\', prediction_column_name]]')
    if func_print : print(f'Return Val :\n{ret_value}')
    return df_realized_vol_per_stock[['row_id', prediction_column_name]]

def past_realized_volatility_per_stock(list_file, prediction_column_name):
    if func_print : print(f'\n\nInside function past_realized_volatility_per_stock \nParameters : list_file = {list_file}, prediction_column_name: {prediction_column_name} ')
    df_past_realized = pd.DataFrame()
    
    for file in tqdm(list_file):
        if func_print : print(f'Inside Loop. for file in tqdm(list_file).\nlist_file : {list_file} \nfile : {file}')
        df_past_realized = pd.concat([df_past_realized, realized_volatility_per_time_id(file, prediction_column_name)])
        if func_print : print('Back to past_realized_volatility_per_stock')
        if func_print : print(f'df_past_realized :\n{df_past_realized}')
    if func_print : print(f'Returning from past_realized_volatility_per_stock. \nret_value :\n{df_past_realized}')
    return df_past_realized

# <font size = 4 color = 'yellow'> Calling function past_realized_volatility_per_stock

In [42]:
df_naive_pred_test = past_realized_volatility_per_stock(list_file = list_order_book_file_test, prediction_column_name = 'pred')



Inside function past_realized_volatility_per_stock 
Parameters : list_file = ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0'], prediction_column_name: pred 


  0%|          | 0/1 [00:00<?, ?it/s]

Inside Loop. for file in tqdm(list_file).
list_file : ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0'] 
file : /kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0


Inside function realized_volatility_per_time_id 
Parameters : file_path = /kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0, prediction_column_name: pred 

df_book_data before calculate_wap 
:    time_id  seconds_in_bucket  bid_price1  ask_price1  bid_price2  ask_price2  bid_size1  ask_size1  bid_size2  ask_size2
0        4                  0    1.000049    1.000590    0.999656    1.000639         91        100        100         24
1        4                  1    1.000049    1.000590    0.999656    1.000639         91        100        100         20
2        4                  5    1.000049    1.000639    0.999656    1.000885        290         20        101         15 



Inside function calculate_wap 
Parameters : df =
  

In [43]:
df_naive_pred_test

,row_id,pred
0,0-4,0.000263


In [44]:
test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
test.head()

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [45]:
df_naive_pred_test.head(100)

,row_id,pred
0,0-4,0.000263


In [46]:
test = test['row_id'].to_frame()
test = test.merge(df_naive_pred_test[['row_id', 'pred']], on = ['row_id'], how = 'left')
test.head()

,row_id,pred
0,0-4,0.000263
1,0-32,NaN
2,0-34,NaN


In [47]:
n_splits = 5
xgb_preds = pd.DataFrame()
cols = ['seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2', 'wap', 'log_return', 'pred']

In [48]:
list_order_trade_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/*')
list_order_trade_file_test = sorted(list_order_trade_file_test, key = lambda x: x.split('=')[1])

In [49]:
for i, (b, t) in tqdm(enumerate(zip(list_order_book_file_test, list_order_trade_file_test)), total = len(list_order_book_file_test)):
    print(i)
    print(b)
    print(t)

  0%|          | 0/1 [00:00<?, ?it/s]

0
/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0
/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0


In [50]:
print(f"list_order_book_file_test : {list_order_book_file_test}")
print(f"list_order_trade_file_test : {list_order_trade_file_test}")

list_order_book_file_test : ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0']
list_order_trade_file_test : ['/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0']


In [51]:
# printing as tuple
for i, m_tuple in enumerate(zip(list_order_book_file_test, list_order_trade_file_test)): 
    print(i)
    print(m_tuple)

0
('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0', '/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0')


In [52]:
# print as individual
for i, (b, t) in enumerate(zip(list_order_book_file_test, list_order_trade_file_test)): 
    print(i)
    print(b) #book
    print(t) #trade

0
/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0
/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0


In [53]:
# adding tqdm
for i, (b, t) in tqdm(enumerate(zip(list_order_book_file_test, list_order_trade_file_test)), total = len(list_order_book_file_test)):
    print(i)
    print(b)
    print(t)

  0%|          | 0/1 [00:00<?, ?it/s]

0
/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0
/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0


In [54]:
# explaining assertion

assert 2 + 2 == 5, "Houston we've got a problem"

AssertionError: Houston we've got a problem

In [64]:
func_print = False

n_splits = 5
xgb_preds = pd.DataFrame()

for i, (b, t) in tqdm(enumerate(zip(list_order_book_file_test, list_order_trade_file_test)), total = len(list_order_book_file_test)):
    
    assert b.split('=')[1] == t.split('=')[1]
    
    stock_id = b.split('=')[1]
    print(f"01. stock_id : {stock_id} \n")
    trade_data = pd.read_parquet(t)
    print(f"02. trade_data :\n{trade_data} \n")
    book_data = pd.read_parquet(b)
    print(f"03. book_data :\n{book_data} \n")
    
    book_data['row_id'] = book_data['time_id'].apply(lambda x:f'{stock_id}-{x}')
    print(f"03. book_data :\n{book_data} \n")
    
    book_data['wap'] = calculate_wap(book_data)
    print(f"04. book_data :\n{book_data} \n")
    
    book_data['log_return'] = book_data.groupby(['time_id'])['wap'].apply(log_return).fillna(0)
    print(f"05. book_data :\n{book_data} \n")
    
    print('\n*********** book_feature ***********')
    ctr = 0
    for trans in ['rank', 'cumcount', 'cummax', 'cummin', 'cumsum']:
        ctr = ctr + 1
        #print(f"\ntrans : {trans}")
        ctr1 = 0
        for feature in ['bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2', 'wap', 'log_return']:
            ctr1 = ctr1 + 1
            book_feature_trans = feature + '_' + trans
            #print(f"book_feature_trans : {book_feature_trans}")
            if i == 0: cols.append(book_feature_trans)
            book_data[book_feature_trans] = book_data.groupby('time_id')[feature].transform(trans)
            print(f"06.{ctr}.{ctr1} book_data[{book_feature_trans}] :\n{book_data[[feature, book_feature_trans]]} \n")
        
    print('\n*********** trade_feature ***********')
    ctr = 0
    for trans in ['mean', 'std', 'max', 'min']:
        ctr = ctr + 1
        ctr1 = 0
        for feature in ['price', 'size', 'order_count']:
            ctr1 = ctr1 + 1
            trade_feature_trans = 'trade_' + feature + '_' + trans
            #print(f"trade_feature_trans : {trade_feature_trans}")
            if i == 0: cols.append(trade_feature_trans)
            trade_data[trade_feature_trans] = trade_data.groupby('time_id')[feature].transform(trans)
            print(f"07.{ctr}.{ctr1} trade_data[{trade_feature_trans}] :\n{trade_data[[feature, trade_feature_trans]]} \n")
            #print(f"trade_data.columns :{str(trade_data.columns.values)} \n")

            book_data = book_data.merge(trade_data[['time_id', trade_feature_trans]].drop_duplicates(), on = 'time_id', how = 'left')
            print(f"book_data.columns :{str(book_data.columns.values)} \n")
            #print(f"08. book_data :\n{book_data.iloc[:, -1]} \n")
    
    print(f'09. test columns : {test.columns.values}\n') 
    print(f'10. book_data columns before merge :\n{book_data.columns.values}\n\n')
    book_data = book_data.merge(test, on = 'row_id', how = 'left', indicator="indicator_column") #indicator indicates rows in left_only, right_only or both 
    print(f'11. book_data columns after merge :\n{book_data.columns.values}\n')
    
    book_data['target'] = 0
    print(f'12. book_data : \n{book_data[['row_id', 'pred', 'indicator_column', 'target']]}')
    
    

SyntaxError: invalid syntax (<ipython-input-64-f9f8bb6c1f81>, line 64)

In [56]:
print(f'test : {test.columns.values}')
print(test)

test : ['row_id' 'pred']
  row_id      pred
0    0-4  0.000263
1   0-32       NaN
2   0-34       NaN


In [57]:
print(f'test : {test.columns.values}') 
print(f'book_data : {book_data.columns.values} \n')
print(book_data['row_id'].head())

book_data : ['time_id' 'seconds_in_bucket' 'bid_price1' 'ask_price1' 'bid_price2' 'ask_price2' 'bid_size1' 'ask_size1' 'bid_size2' 'ask_size2' 'row_id' 'wap' 'log_return' 'bid_price1_rank' 'ask_price1_rank' 'bid_price2_rank' 'ask_price2_rank' 'bid_size1_rank' 'ask_size1_rank' 'bid_size2_rank' 'ask_size2_rank' 'wap_rank' 'log_return_rank' 'bid_price1_cumcount' 'ask_price1_cumcount' 'bid_price2_cumcount' 'ask_price2_cumcount' 'bid_size1_cumcount' 'ask_size1_cumcount' 'bid_size2_cumcount' 'ask_size2_cumcount' 'wap_cumcount' 'log_return_cumcount' 'bid_price1_cummax' 'ask_price1_cummax' 'bid_price2_cummax' 'ask_price2_cummax' 'bid_size1_cummax' 'ask_size1_cummax' 'bid_size2_cummax' 'ask_size2_cummax' 'wap_cummax' 'log_return_cummax' 'bid_price1_cummin' 'ask_price1_cummin' 'bid_price2_cummin' 'ask_price2_cummin' 'bid_size1_cummin' 'ask_size1_cummin' 'bid_size2_cummin' 'ask_size2_cummin' 'wap_cummin' 'log_return_cummin' 'bid_price1_cumsum' 'ask_price1_cumsum' 'bid_price2_cumsum' 'ask_price2_c

In [58]:
print(f'test : {test.columns.values}') 
print(f'book_data columns before merge :\n{book_data.columns.values}\n\n')
book_data = book_data.merge(test, on = 'row_id', how = 'left', indicator="indicator_column") #indicator indicates rows in left_only, right_only or both 
print(f'book_data columns after merge :\n{book_data.columns.values}')

book_data columns before merge :
['time_id' 'seconds_in_bucket' 'bid_price1' 'ask_price1' 'bid_price2' 'ask_price2' 'bid_size1' 'ask_size1' 'bid_size2' 'ask_size2' 'row_id' 'wap' 'log_return' 'bid_price1_rank' 'ask_price1_rank' 'bid_price2_rank' 'ask_price2_rank' 'bid_size1_rank' 'ask_size1_rank' 'bid_size2_rank' 'ask_size2_rank' 'wap_rank' 'log_return_rank' 'bid_price1_cumcount' 'ask_price1_cumcount' 'bid_price2_cumcount' 'ask_price2_cumcount' 'bid_size1_cumcount' 'ask_size1_cumcount' 'bid_size2_cumcount' 'ask_size2_cumcount' 'wap_cumcount' 'log_return_cumcount' 'bid_price1_cummax' 'ask_price1_cummax' 'bid_price2_cummax' 'ask_price2_cummax' 'bid_size1_cummax' 'ask_size1_cummax' 'bid_size2_cummax' 'ask_size2_cummax' 'wap_cummax' 'log_return_cummax' 'bid_price1_cummin' 'ask_price1_cummin' 'bid_price2_cummin' 'ask_price2_cummin' 'bid_size1_cummin' 'ask_size1_cummin' 'bid_size2_cummin' 'ask_size2_cummin' 'wap_cummin' 'log_return_cummin' 'bid_price1_cumsum' 'ask_price1_cumsum' 'bid_price2_

In [59]:
print(book_data[['row_id', 'pred', 'indicator_column']].head())

  row_id      pred indicator_column
0    0-4  0.000263             both
1    0-4  0.000263             both
2    0-4  0.000263             both


In [60]:
book_data['target'] = 0

In [65]:
print('hello')

hello
